In [1]:
# We will test this cascade model
import numpy as np
def GetResult(Pred_Final, TestGT):
    TP = np.sum(np.logical_and(Pred_Final==1, TestGT==1))
    FP = np.sum(np.logical_and(Pred_Final==1, TestGT==0))
    TN = np.sum(np.logical_and(Pred_Final==0, TestGT==0))
    FN = np.sum(np.logical_and(Pred_Final==0, TestGT==1))
    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN)
    return TP, FP, TN, FN, TPR, FPR

In [12]:
print("Testing on training set")
Final_TP, Final_FP, Final_TN, Final_FN = (0,0,0,0)
Training_Mask = np.array([True]*2468) # Initial mask are all True
RealMaskList = []
for Stage in range(3):
    with open("Summarization_Stage%d.txt" %Stage) as f:
        lines = f.readlines()
    T = int(lines[0].split()[1])
    SltWeakCls = [int(x.strip().split()[0]) for x in lines[2:T+2]]
    SltThIdx = [int(x.strip().split()[1]) for x in lines[2:T+2]]
    alpha =  [float(x.strip().split()[2]) for x in lines[2:T+2]]

    # Re-shape the threshold matrix and feature matrix
    HaarResults = np.load("HaarResults.npy")  # HaarResults  166k x 2468
    HaarResults = np.array([HaarResults[x,Training_Mask] for x in SltWeakCls])
    HaarResults = np.hstack([HaarResults, 
                             np.ones((HaarResults.shape[0],1))*1e7])

    TrainHaarResults = np.load("HaarResults.npy")
    TrainHaarResults = np.array([TrainHaarResults[x,:] for x in SltWeakCls])
    TrainGT = np.array([1]*710 + [0]*1758)
    for RealMask0 in RealMaskList:
        TrainGT = TrainGT[RealMask0]
        TrainHaarResults = TrainHaarResults[:, RealMask0]
        
    # Final strong classifier: Use alpha, SltWeakCls, SltThIdx to construct
    #    Eq: np.sum(alpha * h(x)) - 0.5*sum(alpha) >= 0
    # Compute final pred, (2468,) shape 0-1 vector
    # Compute false-postive rate->FP/(FP+TN) should be 30% 
    # Compute true-postive rate->TP/(TP+FN) should be 99% or 1
    Feature_Final = np.zeros(TrainGT.shape)
    Dec_Th = np.sum(alpha)/2
    for t in range(T):
        # In this iteration SltWeakCls[t] is selected, with Th @ SltThIdx[t]
        Feature = TrainHaarResults[t, :]
        Th = HaarResults[t, int(SltThIdx[t])%HaarResults.shape[1]] 
        if(SltThIdx[t] < HaarResults.shape[1]): # Positive polarity
            Pred = (Feature>=Th)
            Feature_Final = Feature_Final + Pred * alpha[t]
        else: # Negative polarity
            Pred = (Feature<Th)
            Feature_Final = Feature_Final + Pred * alpha[t]
    Pred_Final = (Feature_Final > Dec_Th)
    TP, FP, TN, FN, TPR, FPR = GetResult(Pred_Final, TrainGT)
    print("TP, FP, TN, FN, TPR, FPR = (%d, %d, %d, %d, %.4f, %.4f) (In step)" %(TP, FP, TN, FN, TPR, FPR))
    
    # Negative results are finalized
    Final_TN = Final_TN + TN
    Final_FN = Final_FN + FN
    print("TP, FP, TN, FN, TPR, FPR = (%d, %d, %d, %d, %.4f, %.4f) (Cumul.)" 
          %(TP, FP, Final_TN, Final_FN, TP/(TP+Final_FN), FP/(FP+Final_TN)))
    
    # Positive results go to next stage
    Real_Mask = (Pred_Final==1)
    RealMaskList.append(Real_Mask)
    print("Mask of real length %d in this step" %Real_Mask.shape[0])
    Training_Mask = np.load("Mask_epoch_%d.npy" %Stage)
    print("Mask of length %d loaded" %(Training_Mask.shape[0]))   

Final_TP = TP
Final_FP = FP
print("Final: TP, FP, TN, FN, TPR, FPR = (%d, %d, %d, %d, %.4f, %.4f)" 
      %(Final_TP, Final_FP, Final_TN, Final_FN, 
        Final_TP/(Final_TP+Final_FN), Final_FP/(Final_FP+Final_TN)))

Testing on training set
TP, FP, TN, FN, TPR, FPR = (659, 171, 1587, 51, 0.9282, 0.0973) (In step)
TP, FP, TN, FN, TPR, FPR = (659, 171, 1587, 51, 0.9282, 0.0973) (Cumul.)
Mask of real length 2468 in this step
Mask of length 2468 loaded
TP, FP, TN, FN, TPR, FPR = (595, 2, 169, 64, 0.9029, 0.0117) (In step)
TP, FP, TN, FN, TPR, FPR = (595, 2, 1756, 115, 0.8380, 0.0011) (Cumul.)
Mask of real length 830 in this step
Mask of length 2468 loaded
TP, FP, TN, FN, TPR, FPR = (595, 0, 2, 0, 1.0000, 0.0000) (In step)
TP, FP, TN, FN, TPR, FPR = (595, 0, 1758, 115, 0.8380, 0.0000) (Cumul.)
Mask of real length 597 in this step
Mask of length 2468 loaded
Final: TP, FP, TN, FN, TPR, FPR = (595, 0, 1758, 115, 0.8380, 0.0000)


In [13]:
print("Testing on test set")
Final_TP, Final_FP, Final_TN, Final_FN = (0,0,0,0)
Training_Mask = np.array([True]*2468) # Initial mask are all True
RealMaskList = []
for Stage in range(3):
    with open("Summarization_Stage%d.txt" %Stage) as f:
        lines = f.readlines()
    T = int(lines[0].split()[1])
    SltWeakCls = [int(x.strip().split()[0]) for x in lines[2:T+2]]
    SltThIdx = [int(x.strip().split()[1]) for x in lines[2:T+2]]
    alpha =  [float(x.strip().split()[2]) for x in lines[2:T+2]]

    # Re-shape the threshold matrix and feature matrix
    HaarResults = np.load("HaarResults.npy")  # HaarResults  166k x 2468
    HaarResults = np.array([HaarResults[x,Training_Mask] for x in SltWeakCls])
    HaarResults = np.hstack([HaarResults, 
                             np.ones((HaarResults.shape[0],1))*1e7])

    TestHaarResults = np.load("TestHaarResults.npy")
    TestHaarResults = np.array([TestHaarResults[x,:] for x in SltWeakCls])
    TestGT = np.array([1]*178 + [0]*440)
    for RealMask0 in RealMaskList:
        TestGT = TestGT[RealMask0]
        TestHaarResults = TestHaarResults[:, RealMask0]
        
    # Final strong classifier: Use alpha, SltWeakCls, SltThIdx to construct
    #    Eq: np.sum(alpha * h(x)) - 0.5*sum(alpha) >= 0
    # Compute final pred, (2468,) shape 0-1 vector
    # Compute false-postive rate->FP/(FP+TN) should be 30% 
    # Compute true-postive rate->TP/(TP+FN) should be 99% or 1
    Feature_Final = np.zeros(TestGT.shape)
    Dec_Th = np.sum(alpha)/2
    for t in range(T):
        # In this iteration SltWeakCls[t] is selected, with Th @ SltThIdx[t]
        Feature = TestHaarResults[t, :]
        Th = HaarResults[t, int(SltThIdx[t])%HaarResults.shape[1]] 
        if(SltThIdx[t] < HaarResults.shape[1]): # Positive polarity
            Pred = (Feature>=Th)
            Feature_Final = Feature_Final + Pred * alpha[t]
        else: # Negative polarity
            Pred = (Feature<Th)
            Feature_Final = Feature_Final + Pred * alpha[t]
    Pred_Final = (Feature_Final > Dec_Th)
    TP, FP, TN, FN, TPR, FPR = GetResult(Pred_Final, TestGT)
    print("TP, FP, TN, FN, TPR, FPR = (%d, %d, %d, %d, %.4f, %.4f) (In step)" %(TP, FP, TN, FN, TPR, FPR))
    
    # Negative results are finalized
    Final_TN = Final_TN + TN
    Final_FN = Final_FN + FN
    print("TP, FP, TN, FN, TPR, FPR = (%d, %d, %d, %d, %.4f, %.4f) (Cumul.)" 
          %(TP, FP, Final_TN, Final_FN, TP/(TP+Final_FN), FP/(FP+Final_TN)))
    
    # Positive results go to next stage
    Real_Mask = (Pred_Final==1)
    RealMaskList.append(Real_Mask)
    print("Mask of real length %d in this step" %Real_Mask.shape[0])
    Training_Mask = np.load("Mask_epoch_%d.npy" %Stage)
    print("Mask of length %d loaded" %(Training_Mask.shape[0]))   

Final_TP = TP
Final_FP = FP
print("Final: TP, FP, TN, FN, TPR, FPR = (%d, %d, %d, %d, %.4f, %.4f)" 
      %(Final_TP, Final_FP, Final_TN, Final_FN, 
        Final_TP/(Final_TP+Final_FN), Final_FP/(Final_FP+Final_TN)))

Testing on test set
TP, FP, TN, FN, TPR, FPR = (154, 51, 389, 24, 0.8652, 0.1159) (In step)
TP, FP, TN, FN, TPR, FPR = (154, 51, 389, 24, 0.8652, 0.1159) (Cumul.)
Mask of real length 618 in this step
Mask of length 2468 loaded
TP, FP, TN, FN, TPR, FPR = (136, 8, 43, 18, 0.8831, 0.1569) (In step)
TP, FP, TN, FN, TPR, FPR = (136, 8, 432, 42, 0.7640, 0.0182) (Cumul.)
Mask of real length 205 in this step
Mask of length 2468 loaded
TP, FP, TN, FN, TPR, FPR = (136, 8, 0, 0, 1.0000, 1.0000) (In step)
TP, FP, TN, FN, TPR, FPR = (136, 8, 432, 42, 0.7640, 0.0182) (Cumul.)
Mask of real length 144 in this step
Mask of length 2468 loaded
Final: TP, FP, TN, FN, TPR, FPR = (136, 8, 432, 42, 0.7640, 0.0182)
